<a href="https://colab.research.google.com/github/cjprybol/Mycelia/blob/master/COLAB_TEMPLATE_julia_with_conda_and_rclone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import io
import os
from google.colab import userdata
import subprocess

In [ ]:
# first, install rclone to connect back and forth with Google Drive
# since in the colab google drive integration doesn't work with non-python kernels
!sudo -v ; curl https://rclone.org/install.sh | sudo bash

rclone_conf_content = f'''
[my_drive]
type = drive
scope = drive
token = {userdata.get('RCLONE_CONFIG_PERSONAL_GOOGLE_DRIVE_TOKEN')}
client_id = {userdata.get('RCLONE_CONFIG_MY_DRIVE_CLIENT_ID')}
client_secret = {userdata.get('RCLONE_CONFIG_MY_DRIVE_CLIENT_SECRET')}
team_drive =
'''
rclone_conf_file = '/root/.config/rclone/rclone.conf'
rclone_dir = os.path.dirname(rclone_conf_file)
!mkdir -p {rclone_dir}
with io.open(rclone_conf_file, 'w', encoding='utf-8') as f:
  f.write(rclone_conf_content)

def run_command_and_print_subset(command):
    """
    Runs a shell command, captures the output, and prints the first 3, last 3,
    and middle 3 items with dots for gaps. Includes a summary of total items.

    Args:
        command: The shell command to execute.
    """
    print(f"Running command: {command}")  # Print the command being run
    process = subprocess.run(command, shell=True, capture_output=True, text=True)
    output = process.stdout.strip().split('\n')
    num_items = len(output)

    print(f"Total items: {num_items}")  # Print the total number of items

    if num_items >= 9:  # Check if there are enough items for middle 3
        print('\n'.join(output[:3]))
        print("\u22EE")  # Dots to indicate a gap
        print('\n'.join(output[num_items // 2 - 1:num_items // 2 + 2]))  # Middle 3
        print("\u22EE")  # Dots to indicate a gap
        print('\n'.join(output[-3:]))
    else:
        print('\n'.join(output))

run_command_and_print_subset('rclone lsf my_drive:')

In [ ]:
# initial creation
# ssh-keygen -t ed25519 -C "cameron.prybol@gmail.com"

normalized_ssh_key = userdata.get('ssh_key').replace('\r\n', '\n').replace('\r', '\n')
ssh_key_content = f'''{normalized_ssh_key}
'''
ssh_key_file = '/root/.ssh/id_ed25519'
ssh_key_dir = os.path.dirname(ssh_key_file)
!mkdir -p {ssh_key_dir}
with io.open(ssh_key_file, 'w') as f:
  f.write(ssh_key_content)
!chmod 600 {ssh_key_file}

normalized_ssh_key_pub = userdata.get('ssh_pub_key').replace('\r\n', '\n').replace('\r', '\n')
ssh_key_pub_content = f'''{normalized_ssh_key_pub}
'''
ssh_key_pub_file = '/root/.ssh/id_ed25519.pub'
with io.open(ssh_key_pub_file, 'w') as f:
  f.write(ssh_key_pub_content)

In [ ]:
# activate ssh keys
!eval "$(ssh-agent -s)"
!ssh-add /root/.ssh/id_ed25519

!git config --global user.email "cameron.prybol@gmail.com"
!git config --global user.name "Cameron Prybol"
# write ssh keys from environment secrets to files

# $HOME/workspace is preferred default working directory on other jupyter systems
# /content is default working directory for file tree in Colab
# symbolically link /content to $HOME/workspace to enable ease of code-reuse across systems
!cd /content && rm -rf sample_data
!ln -s /content "$HOME/workspace"
!cd "$HOME/workspace"

# add github to known hosts to allow automatic cloning without interactive confirmation
!ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts
!git clone git@github.com:cjprybol/Mycelia.git

In [ ]:
# # # download Julia installer
# !wget https://raw.githubusercontent.com/abelsiqueira/jill/main/jill.sh
# # Install Julia LTS (does autolink)
# !bash jill.sh --lts --yes
# Install julia with Juliaup installer
!curl -fsSL https://install.julialang.org | sh -s -- --yes --default-channel lts
# doesn't automatically link
!ln -s /root/.juliaup/bin/julia /usr/bin/julia
!julia -e 'using Pkg; Pkg.add("IJulia"); import IJulia; IJulia.installkernel("julia", env=Dict("JULIA_NUM_THREADS"=>"$(Sys.CPU_THREADS)"))'
# 3. hot-fix patch to strip the version suffix of the installed kernel so that this notebook kernelspec is version agnostic
!jupyter kernelspec install $(jupyter kernelspec list | grep julia | tr -s ' ' | cut -d' ' -f3) --replace --name julia

# !Activate Julia environment!
1. Ensure that runtime is set to Julia LTS Version
2. Save your notebook to update default notebook runtime
3. Refresh page to switch to Julia kernel

In [ ]:
import Pkg
Pkg.activate(;temp=true)
Pkg.add("Revise")
import Revise

# you can comment these all and uncomment them one-by-one as you need them
# to avoid loading more packages than you need
pkgs = [
    "AlgebraOfGraphics",
    "Arrow",
    "BioAlignments",
    "BioSequences",
    "BioSymbols",
    "CairoMakie",
    "Clustering",
    "CodecZlib",
    "Colors",
    "ColorSchemes",
    "Conda",
    "CSV",
    "DataFrames",
    "DataStructures",
    "Dates",
    "DelimitedFiles",
    "Distances",
    "Distributions",
    "DocStringExtensions",
    "Downloads",
    "FASTX",
    "FileIO",
    "GenomicAnnotations",
    "GeoMakie",
    "GFF3",
    "GLM",
    "GraphMakie",
    "Graphs",
    "HDF5",
    "HTTP",
    "JLD2",
    "JSON",
    "Karnak",
    "Kmers",
    "LsqFit",
    "Luxor",
    "Makie",
    "MetaGraphs",
    "Mmap",
    "OrderedCollections",
    "Plots",
    "Primes",
    "ProgressMeter",
    "Random",
    "SparseArrays",
    "Statistics",
    "StatsBase",
    "StatsPlots",
    "TopoPlots",
    "uCSV",
    "XAM",
    "XMLDict"
]
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

Pkg.develop(path="$(homedir())/workspace/Mycelia")
import Mycelia
Mycelia.system_overview()